In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt

# 配色常量
PURPLE = '#2D22FF'
PURPLE_LIGHT = '#C7C5FF'
PURPLE_DARK = '#1B1499'

BLUE = '#00AAEE'
BLUE_LIGHT = '#B7F2FF'
BLUE_DARK = '#0077A7'

GRAY = '#404040'

# 读取实验数据
file_path = 'ETTm1_robustness.txt'
with open(file_path, 'r') as f:
    lines = f.readlines()

data = []
current_config = {}

for line in lines:
    line = line.strip()
    if not line:
        continue
    
    if line.startswith('ETTm1_'):
        # 解析配置
        pl_match = re.search(r'pl:(\d+)', line)
        nl_match = re.search(r'nl:([\d\.]+)', line)
        
        pl = int(pl_match.group(1)) if pl_match else None
        nl = float(nl_match.group(1)) if nl_match else 0.0
        
        current_config = {'pl': pl, 'nl': nl}
        
    elif line.startswith('mse:'):
        # 解析结果
        parts = line.split(',')
        mse = float(parts[0].split(':')[1])
        mae = float(parts[1].split(':')[1])
        
        entry = current_config.copy()
        entry['mse'] = mse
        entry['mae'] = mae
        data.append(entry)

df = pd.DataFrame(data)
# 确保按照预测长度排序
df = df.sort_values(by=['pl', 'nl'])
print("读取到的原始数据前几行：")
print(df.head())

In [ ]:
# 计算百分比变化并生成表格
metrics = ['mse', 'mae']
noise_levels = [0.1, 0.2, 0.3, 0.5]
base_noise = 0.0

results = []

# 获取所有唯一的预测长度
prediction_lengths = sorted(df['pl'].unique())

for pl in prediction_lengths:
    row = {'Prediction Length': pl}
    
    # 获取基准值 (noise level = 0)
    base_data = df[(df['pl'] == pl) & (df['nl'] == base_noise)]
    if base_data.empty:
        continue
        
    base_mse = base_data['mse'].values[0]
    base_mae = base_data['mae'].values[0]
    
    row[f'MSE (nl={base_noise})'] = round(base_mse, 3)
    row[f'MAE (nl={base_noise})'] = round(base_mae, 3)
    
    for nl in noise_levels:
        curr_data = df[(df['pl'] == pl) & (df['nl'] == nl)]
        if curr_data.empty:
            continue
            
        curr_mse = curr_data['mse'].values[0]
        curr_mae = curr_data['mae'].values[0]
        
        row[f'MSE (nl={nl})'] = round(curr_mse, 3)
        row[f'MAE (nl={nl})'] = round(curr_mae, 3)
        
        # 计算百分比变化
        mse_pct_change = ((curr_mse - base_mse) / base_mse) * 100
        mae_pct_change = ((curr_mae - base_mae) / base_mae) * 100
        
        row[f'MSE % Change (nl={nl})'] = round(mse_pct_change, 3)
        row[f'MAE % Change (nl={nl})'] = round(mae_pct_change, 3)
        
    results.append(row)

results_df = pd.DataFrame(results)
results_df.set_index('Prediction Length', inplace=True)

# 显示表格
print("实验结果及百分比变化表：")
display(results_df)

In [ ]:
# 绘制百分比变化折线图
plt.figure(figsize=(14, 6))

# MSE Percentage Change
plt.subplot(1, 2, 1)
# 生成紫色渐变
mse_colors = ['#C7C5FF', '#9490FF', '#605CFF', '#2D22FF']

for i, nl in enumerate(noise_levels):
    col_name = f'MSE % Change (nl={nl})'
    if col_name in results_df.columns:
        c = mse_colors[i] if i < len(mse_colors) else PURPLE
        plt.plot(results_df.index, results_df[col_name], marker='o', label=f'Noise Level {nl}', color=c)

plt.title('MSE Percentage Change vs Prediction Length')
plt.xlabel('Prediction Length')
plt.ylabel('Percentage Change (%)')
plt.legend()
plt.grid(True)

# MAE Percentage Change
plt.subplot(1, 2, 2)
# 生成蓝色渐变
mae_colors = ['#B7F2FF', '#7AD7FF', '#33BBF7', '#00AAEE']

for i, nl in enumerate(noise_levels):
    col_name = f'MAE % Change (nl={nl})'
    if col_name in results_df.columns:
        c = mae_colors[i] if i < len(mae_colors) else BLUE
        plt.plot(results_df.index, results_df[col_name], marker='o', label=f'Noise Level {nl}', color=c)

plt.title('MAE Percentage Change vs Prediction Length')
plt.xlabel('Prediction Length')
plt.ylabel('Percentage Change (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

# 绘制并排的直方图 (Bar Charts)
# 每个预测长度一个子图
# X轴: Noise Level
# 左Y轴: MSE/MAE (Bar)
# 右Y轴: Change Rate % (Line)

prediction_lengths = sorted(df['pl'].unique())
n_pl = len(prediction_lengths)
all_noise_levels = [0.0, 0.1, 0.2, 0.3, 0.5]

# 设置图形大小 - 缩小高度
fig, axes = plt.subplots(1, n_pl, figsize=(6*n_pl, 5), sharey=False)
if n_pl == 1:
    axes = [axes]

for i, pl in enumerate(prediction_lengths):
    ax = axes[i]
    
    # 准备数据
    mses = []
    maes = []
    valid_levels = []
    mse_changes = []
    mae_changes = []
    
    # 获取基准值 (nl=0)
    base_row = df[(df['pl'] == pl) & (df['nl'] == 0.0)]
    base_mse = base_row['mse'].values[0] if not base_row.empty else None
    base_mae = base_row['mae'].values[0] if not base_row.empty else None
    
    for nl in all_noise_levels:
        # 查找对应 pl 和 nl 的数据
        row = df[(df['pl'] == pl) & (df['nl'] == nl)]
        if not row.empty:
            curr_mse = row['mse'].values[0]
            curr_mae = row['mae'].values[0]
            mses.append(curr_mse)
            maes.append(curr_mae)
            valid_levels.append(nl)
            
            # 计算变化率
            if base_mse:
                mse_changes.append(((curr_mse - base_mse) / base_mse) * 100)
            else:
                mse_changes.append(0)
                
            if base_mae:
                mae_changes.append(((curr_mae - base_mae) / base_mae) * 100)
            else:
                mae_changes.append(0)
        else:
            # 如果缺失数据，填0
            mses.append(0)
            maes.append(0)
            valid_levels.append(nl)
            mse_changes.append(0)
            mae_changes.append(0)
            
    # 设置柱状图位置
    x = np.arange(len(valid_levels))
    width = 0.25  # 缩小宽度
    
    # 绘制柱状图 - 使用浅色填充，深色描边
    rects1 = ax.bar(x - width/2, mses, width, label='MSE', color=PURPLE_LIGHT, edgecolor=PURPLE, linewidth=1.5)
    rects2 = ax.bar(x + width/2, maes, width, label='MAE', color=BLUE_LIGHT, edgecolor=BLUE, linewidth=1.5)
    
    # 设置左侧Y轴范围 - 动态调整
    all_vals = [v for v in mses + maes if v > 0]
    if all_vals:
        y_min = min(all_vals)
        y_max = max(all_vals)
        ax.set_ylim(y_min * 0.85, y_max * 1.15)

    # 设置标签和标题 - 增大字体
    ax.set_title(f'Prediction Length: {pl}', fontsize=16, fontweight='bold', color=GRAY)
    ax.set_xlabel('Std Dev', fontsize=14, color=GRAY)
    ax.set_ylabel('Error Value', fontsize=14, color=GRAY)
    ax.set_xticks(x)
    ax.set_xticklabels(valid_levels, fontsize=12)
    ax.tick_params(axis='y', labelsize=12, colors=GRAY)
    
    # 美化左侧坐标轴
    ax.tick_params(axis='x', colors=GRAY)
    for spine in ax.spines.values():
        spine.set_edgecolor(GRAY)
    
    # 创建右侧Y轴绘制变化率折线图
    ax2 = ax.twinx()
    # 绘制折线 - 使用更深的颜色
    line1 = ax2.plot(x, mse_changes, color=PURPLE_DARK, marker='o', linestyle='-', linewidth=2, label='MSE % Change')
    line2 = ax2.plot(x, mae_changes, color=BLUE_DARK, marker='s', linestyle='-', linewidth=2, label='MAE % Change')
    
    ax2.set_ylabel('Change Rate (%)', fontsize=14, color=GRAY)
    ax2.tick_params(axis='y', labelsize=12, colors=GRAY)
    ax2.spines['right'].set_color(GRAY)
    ax2.spines['left'].set_color(GRAY)
    ax2.spines['top'].set_color(GRAY)
    ax2.spines['bottom'].set_color(GRAY)
    
    # 合并图例
    lines = [rects1, rects2] + line1 + line2
    labels = [l.get_label() for l in lines]
    ax.legend(lines, labels, loc='upper left', fontsize=10)
    
    ax.grid(True, axis='y', linestyle='--', alpha=0.3, color=GRAY)

plt.tight_layout()
plt.show()